# MTA 1 Exercise

In [112]:
import pandas as pd
import numpy as np

## Exercise 1

- Open up a new IPython notebook
- Download MTA turnstile data files for the following three dates: ```160903, 160910, 160917```

- Load the files into a pandas DataFrame (hint: `pd.read_csv()` to load files and `pd.concat()` to combine DataFrames)


In [113]:
one = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_160903.txt")
two = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_160910.txt")
three = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_160917.txt")

In [114]:
one

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041
1,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,04:00:00,REGULAR,5799463,1966044
2,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,08:00:00,REGULAR,5799492,1966079
3,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,12:00:00,REGULAR,5799610,1966155
4,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,16:00:00,REGULAR,5799833,1966214
...,...,...,...,...,...,...,...,...,...,...,...
193134,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/02/2016,05:00:00,REGULAR,5554,259
193135,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/02/2016,09:00:00,REGULAR,5554,259
193136,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/02/2016,13:00:00,REGULAR,5554,259
193137,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/02/2016,17:00:00,REGULAR,5554,259


In [115]:
df = pd.concat([one, two, three], join="inner", ignore_index=True)

In [116]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041
1,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,04:00:00,REGULAR,5799463,1966044
2,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,08:00:00,REGULAR,5799492,1966079
3,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,12:00:00,REGULAR,5799610,1966155
4,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,16:00:00,REGULAR,5799833,1966214


## Exercise 2

- Let's turn this into a time series.

- Our pandas dataframe has columns called `Date` and `Time` (what datatype did pandas assign to these columns on import?). However, in python and pandas we can convert date and time information to _datetime_ objects, which allow us to do time-based operations

- Using either [pd.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) in pandas or the [python datetime library](https://docs.python.org/2/library/datetime.html), combine the `Date` and `Time` columns into a single new column of the datetime datatype

In [117]:
df.dtypes

C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                  int64
EXITS                                                                    int64
dtype: object

Pandas assigned an `object` (i.e. string) type to the `DATE` and `TIME` columns on import.

To begin with for data cleaning, I will clean the column names to make it easier to wrangle later on.

In [118]:
df.columns = [column.strip().lower().replace('/','_') for column in df.columns] 
df.columns

Index(['c_a', 'unit', 'scp', 'station', 'linename', 'division', 'date', 'time',
       'desc', 'entries', 'exits'],
      dtype='object')

In [119]:
df['date_time'] = pd.to_datetime(df.date + ' ' + df.time)

In [120]:
df.dtypes

c_a                  object
unit                 object
scp                  object
station              object
linename             object
division             object
date                 object
time                 object
desc                 object
entries               int64
exits                 int64
date_time    datetime64[ns]
dtype: object

The `date_time` column is now a datetime object.

## Exercise 3a
* Each combination of C/A, UNIT, SCP, and STATION represents a unique turnstile. Take a look at one specific turnstile on a specific date. What does each row in the dataframe represent?
* Obtain the maximum ENTRIES value for each day, for each unique turnstile.

To look at a specifc turnstile, I'll use a mask. 

In [121]:
mask = ((df["c_a"] == "A002") &
        (df["unit"] == "R051") & 
        (df["scp"] == "02-00-00") & 
        (df["station"] == "59 ST"))

df[mask].head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041,2016-08-27 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,04:00:00,REGULAR,5799463,1966044,2016-08-27 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,08:00:00,REGULAR,5799492,1966079,2016-08-27 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,12:00:00,REGULAR,5799610,1966155,2016-08-27 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,16:00:00,REGULAR,5799833,1966214,2016-08-27 16:00:00


It looks like, for a unique turnstile, each row represents a 4 hour window for a specific day with the total number of CUMULATIVE entries and exits at that point. 

As per the question, let's filter for the maximum entries in a four hour window per day and call that new dataframe `df_daily`. The last reading for each day is the maximum number of entries, therefore we would filter for the last row in each day. 

In [122]:
df_daily = (df
            .groupby(["c_a", "unit", "scp", "station", "date"],as_index=False)
            .entries.last())

In [123]:
df_daily

,c_a,unit,scp,station,date,entries
0,A002,R051,02-00-00,59 ST,08/27/2016,5800121
1,A002,R051,02-00-00,59 ST,08/28/2016,5800798
2,A002,R051,02-00-00,59 ST,08/29/2016,5802336
3,A002,R051,02-00-00,59 ST,08/30/2016,5803875
4,A002,R051,02-00-00,59 ST,08/31/2016,5805383
...,...,...,...,...,...,...
95997,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/12/2016,5554
95998,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/13/2016,5554
95999,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/14/2016,5554
96000,TRAM2,R469,00-05-01,RIT-ROOSEVELT,09/15/2016,5554


## Exercise 3b

- Work off of the daily maximum `ENTRIES` calculations from Problem 3a. Recall that the `ENTRIES` column contains cumulative entries on each day. We would now like you to calculate daily entries, i.e. the number of new entries gained each day
- Hint: Group the data by turnstile and use the Pandas `.apply()` method to compute the same differencing function for each turnstile. Check out the `.shift()` and `.diff()` DataFrame methods for this purpose
- Look through your results. Do they all make sense?

To calculate daily entries, I have to deduct the former day's cumulative entries from each row's `entries` column. I can do this by using `.apply()` to apply a single function for each turnstile.

In [124]:
df_daily[['prev_date', 'prev_entries']] = (df_daily
                                   .groupby(["c_a", "unit", "scp", "station"])["date", "entries"]
                                   .apply(lambda x : x.shift(1)))

/var/folders/kg/3y370xfs2tz2m6sdttj9x94m0000gn/T/ipykernel_22451/1636221945.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_daily[['prev_date', 'prev_entries']] = (df_daily


In [125]:
df_daily.head()

,c_a,unit,scp,station,date,entries,prev_date,prev_entries
0,A002,R051,02-00-00,59 ST,08/27/2016,5800121,NaN,NaN
1,A002,R051,02-00-00,59 ST,08/28/2016,5800798,08/27/2016,5800121.0
2,A002,R051,02-00-00,59 ST,08/29/2016,5802336,08/28/2016,5800798.0
3,A002,R051,02-00-00,59 ST,08/30/2016,5803875,08/29/2016,5802336.0
4,A002,R051,02-00-00,59 ST,08/31/2016,5805383,08/30/2016,5803875.0


I will remove the NaN value for the first row for each turnstile.

In [126]:
df_daily.dropna(subset=['prev_date'], inplace=True)

In [127]:
df_daily.head()

,c_a,unit,scp,station,date,entries,prev_date,prev_entries
1,A002,R051,02-00-00,59 ST,08/28/2016,5800798,08/27/2016,5800121.0
2,A002,R051,02-00-00,59 ST,08/29/2016,5802336,08/28/2016,5800798.0
3,A002,R051,02-00-00,59 ST,08/30/2016,5803875,08/29/2016,5802336.0
4,A002,R051,02-00-00,59 ST,08/31/2016,5805383,08/30/2016,5803875.0
5,A002,R051,02-00-00,59 ST,09/01/2016,5806990,08/31/2016,5805383.0


In [129]:
df_daily['daily_entries'] = df_daily['entries'] - df_daily['prev_entries']

In [130]:
df_daily.head()

,c_a,unit,scp,station,date,entries,prev_date,prev_entries,daily_entries
1,A002,R051,02-00-00,59 ST,08/28/2016,5800798,08/27/2016,5800121.0,677.0
2,A002,R051,02-00-00,59 ST,08/29/2016,5802336,08/28/2016,5800798.0,1538.0
3,A002,R051,02-00-00,59 ST,08/30/2016,5803875,08/29/2016,5802336.0,1539.0
4,A002,R051,02-00-00,59 ST,08/31/2016,5805383,08/30/2016,5803875.0,1508.0
5,A002,R051,02-00-00,59 ST,09/01/2016,5806990,08/31/2016,5805383.0,1607.0


Now let's see if there's anything funky with the `daily_entries` column.

In [139]:
(df_daily[df_daily['daily_entries'] < 0]
 .groupby(['c_a', 'unit', 'scp', 'station'])).head()

,c_a,unit,scp,station,date,entries,prev_date,prev_entries,daily_entries
715,A011,R080,01-00-00,57 ST-7 AV,08/28/2016,887688133,08/27/2016,8.876896e+08,-1475.0
716,A011,R080,01-00-00,57 ST-7 AV,08/29/2016,887685714,08/28/2016,8.876881e+08,-2419.0
717,A011,R080,01-00-00,57 ST-7 AV,08/30/2016,887683207,08/29/2016,8.876857e+08,-2507.0
718,A011,R080,01-00-00,57 ST-7 AV,08/31/2016,887680260,08/30/2016,8.876832e+08,-2947.0
719,A011,R080,01-00-00,57 ST-7 AV,09/01/2016,887677354,08/31/2016,8.876803e+08,-2906.0
...,...,...,...,...,...,...,...,...,...
94554,R646,R110,01-00-01,FLATBUSH AV-B.C,08/28/2016,1508334747,08/27/2016,1.508335e+09,-400.0
94555,R646,R110,01-00-01,FLATBUSH AV-B.C,08/29/2016,1508332956,08/28/2016,1.508335e+09,-1791.0
94556,R646,R110,01-00-01,FLATBUSH AV-B.C,08/30/2016,1508330989,08/29/2016,1.508333e+09,-1967.0
94557,R646,R110,01-00-01,FLATBUSH AV-B.C,08/31/2016,1508329012,08/30/2016,1.508331e+09,-1977.0


Looks like there are negative values for `daily_entries`. Let's take a look at one of these days with negative `daily_entries` values and see what is happening.

In [146]:
mask2 = ((df["c_a"] == "A011") &
        (df["unit"] == "R080") & 
        (df["scp"] == "01-00-00") & 
        (df["station"] == "57 ST-7 AV") &
        (df["date"] == "08/28/2016"))

df[mask2]

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
1447,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,00:00:00,REGULAR,887689248,493418842,2016-08-28 00:00:00
1448,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,04:00:00,REGULAR,887689102,493418751,2016-08-28 04:00:00
1449,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,08:00:00,REGULAR,887689047,493418585,2016-08-28 08:00:00
1450,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,12:00:00,REGULAR,887688820,493418160,2016-08-28 12:00:00
1451,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,16:00:00,REGULAR,887688500,493417607,2016-08-28 16:00:00
1452,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,20:00:00,REGULAR,887688133,493417232,2016-08-28 20:00:00


In this case, we can see that the `entries` counter is working in reverse, for some reason. Let's check out the consecutive day for this case to see what's happening.

In [149]:
mask3 = ((df["c_a"] == "A011") &
        (df["unit"] == "R080") & 
        (df["scp"] == "01-00-00") & 
        (df["station"] == "57 ST-7 AV") &
        ((df["date"] == "08/28/2016") |
        (df["date"] == "08/29/2016")))

df[mask3]

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits,date_time
1447,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,00:00:00,REGULAR,887689248,493418842,2016-08-28 00:00:00
1448,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,04:00:00,REGULAR,887689102,493418751,2016-08-28 04:00:00
1449,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,08:00:00,REGULAR,887689047,493418585,2016-08-28 08:00:00
1450,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,12:00:00,REGULAR,887688820,493418160,2016-08-28 12:00:00
1451,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,16:00:00,REGULAR,887688500,493417607,2016-08-28 16:00:00
1452,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/28/2016,20:00:00,REGULAR,887688133,493417232,2016-08-28 20:00:00
1453,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/29/2016,00:00:00,REGULAR,887687859,493417025,2016-08-29 00:00:00
1454,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/29/2016,04:00:00,REGULAR,887687796,493416991,2016-08-29 04:00:00
1455,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/29/2016,08:00:00,REGULAR,887687693,493416556,2016-08-29 08:00:00
1456,A011,R080,01-00-00,57 ST-7 AV,NQR,BMT,08/29/2016,12:00:00,REGULAR,887687246,493415359,2016-08-29 12:00:00


The counter seems to be working in reverse, so we can just take the negative value of the `daily_entries`